In [1]:
# import pandas as pd
# import matplotlib.pyplot as plt
# import numpy as np
# import re
# from bs4 import BeautifulSoup
# import requests as req
# import json
# from tqdm import tqdm
# import itertools
# from unidecode import unidecode
# import time 

from tqdm.notebook  import tqdm
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

tqdm.pandas()

# IMDb Awards

In [2]:

driver = webdriver.Edge()
driver.get("https://www.imdb.com/event/ev0000147/2023/1")


id=driver.find_elements(By.CLASS_NAME, "event-widgets__award")[0].find_elements(By.CLASS_NAME, "event-widgets__award-nomination")[0].find_element(By.CLASS_NAME, "event-widgets__primary-nominees [href]").get_attribute('href').split('/')[4]
print("imdb id :",id)
driver.close()

imdb id : tt17009710


In [3]:
def is_winner(nominee):
    try:
        nominee.find_element(By.CLASS_NAME, "event-widgets__winner-badge")
        return True
    except:
        return False

In [4]:
def is_name(nominee):
    return 'name' in nominee.get_attribute('href')[:25]

In [5]:
def has_categories(award):
    try:
        award.find_element(By.CLASS_NAME, "event-widgets__award-categories")
        return True
    except:
        return False

In [6]:
def get_awards(url):
    driver = webdriver.Edge()
    driver.get(url)
    try:
        award_list=driver.find_elements(By.CLASS_NAME, "event-widgets__award")

        awards=dict()
        for award in tqdm(award_list):

            award_subcategory=award.find_elements(By.CLASS_NAME, "event-widgets__award-category")

            for sub_award in award_subcategory: # if there is no subcategory this loop will execute only one time normally
                if len(award_subcategory)>1:
                    award_name=award.find_element(By.CLASS_NAME, "event-widgets__award-name").text+' - '+sub_award.find_element(By.CLASS_NAME,"event-widgets__award-category-name").text
                else:
                    award_name=award.find_element(By.CLASS_NAME, "event-widgets__award-name").text
                nominees_list=[]
                for nominee in sub_award.find_elements(By.CLASS_NAME, "event-widgets__award-nomination"):
                    
                    temp_nominee_dict=dict()
                    primary=nominee.find_element(By.CLASS_NAME, "event-widgets__primary-nominees [href]")
                    secondary=nominee.find_element(By.CLASS_NAME, "event-widgets__secondary-nominees") # here we don't need the [href] as it's enough to test the primary and sometimes there is only the primary

                    if is_name(primary): # tests if the primary component is a name of a person or title of a film using the provided url
                        temp_nominee_dict["director/name"]=primary.text
                        temp_nominee_dict["title"]=secondary.text
                        temp_nominee_dict["original_title"]=nominee.find_element(By.CLASS_NAME, "event-widgets__original-title--secondary").text.removesuffix(" (original title)")
                        if len(secondary.text)>0:
                            temp_nominee_dict["imdb_id"]=nominee.find_element(By.CLASS_NAME, "event-widgets__secondary-nominees [href]").get_attribute('href').split('/')[4]
                        else:
                            temp_nominee_dict["imdb_id"]=''
                    else:
                        temp_nominee_dict["director/name"]=secondary.text
                        temp_nominee_dict["title"]=primary.text
                        temp_nominee_dict["original_title"]=nominee.find_element(By.CLASS_NAME, "event-widgets__original-title--primary").text.removesuffix(" (original title)")
                        temp_nominee_dict["imdb_id"]=primary.get_attribute('href').split('/')[4]

                    temp_nominee_dict["winner"]=is_winner(nominee)
                    nominees_list.append(temp_nominee_dict)
                awards[award_name]=nominees_list

        driver.close()
        return awards
    
    except Exception as e:
        driver.close()
        print(f"error : {e}")

    

In [7]:
get_awards("https://www.imdb.com/event/ev0000091/2020/1")

  0%|          | 0/35 [00:00<?, ?it/s]

{'Golden Berlin Bear - Best Film': [{'director/name': 'Mohammad Rasoulof',
   'title': 'There Is No Evil',
   'original_title': 'Sheytan Vojood Nadarad',
   'imdb_id': 'tt11697844',
   'winner': True},
  {'director/name': 'Marco Dutra, Caetano Gotardo',
   'title': 'All the Dead Ones',
   'original_title': 'Todos os Mortos',
   'imdb_id': 'tt8845322',
   'winner': False},
  {'director/name': "Damiano D'Innocenzo, Fabio D'Innocenzo",
   'title': 'Bad Tales',
   'original_title': 'Favolacce',
   'imdb_id': 'tt8526370',
   'winner': False},
  {'director/name': 'Burhan Qurbani',
   'title': 'Berlin Alexanderplatz',
   'original_title': '',
   'imdb_id': 'tt6470924',
   'winner': False},
  {'director/name': 'Ilya Khrzhanovskiy, Jekaterina Oertel',
   'title': 'DAU. Natasha',
   'original_title': '',
   'imdb_id': 'tt11697832',
   'winner': False},
  {'director/name': 'Tsai Ming-liang',
   'title': 'Days',
   'original_title': 'Rizi',
   'imdb_id': 'tt11697820',
   'winner': False},
  {'dire

# next